In [1]:
import os
import datasets

c:\Users\calic\Downloads\imda_data_prep\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
data = datasets.load_dataset('./loadingScript_imda_part3.py', 'all', split="train", streaming=True)

print(next(iter(data)))
print(next(iter(data)))

{'audio': {'path': 'C:\\Users\\calic\\Downloads\\huggingface-dataset\\imda-dataset\\IMDA - National Speech Corpus\\PART3\\tmp_clip.wav', 'array': array([-6.1035156e-05, -3.0517578e-05,  0.0000000e+00, ...,
        3.0517578e-05,  3.0517578e-05, -9.1552734e-05], dtype=float32), 'sampling_rate': 16000}, 'transcript': 'start:0, end:24.718', 'mic': None, 'audio_name': None}
{'audio': {'path': 'C:\\Users\\calic\\Downloads\\huggingface-dataset\\imda-dataset\\IMDA - National Speech Corpus\\PART3\\tmp_clip.wav', 'array': array([-6.1035156e-05, -3.0517578e-05,  0.0000000e+00, ...,
        3.0517578e-05,  3.0517578e-05, -9.1552734e-05], dtype=float32), 'sampling_rate': 16000}, 'transcript': 'start:0, end:24.718', 'mic': None, 'audio_name': None}


In [26]:
from IPython.display import Audio, display

index = 0
for i in data.shuffle():
    if index == 1: 
        break
    display_data = i['audio']

    print(f'sampling rate: {display_data["sampling_rate"]}')
    display(Audio(display_data["array"], rate=display_data["sampling_rate"]))
    print(f'transcript: {i["transcript"]}')
    # print(f'gender: {i["gender"]}')
    # print(f'race: {i["race"]}')
    index+=1

IndexError: list index out of range

In [ ]:
import glob
path_to_data = 'D:\IMDA - National Speech Corpus\PART1'
mic = "CHANNEL0"
speaker = '0001'
metadata_path = os.path.join(path_to_data, "DATA", mic, "SCRIPT", mic[-1]+speaker+'*.TXT')
print(metadata_path)
script_list = glob.glob(metadata_path)
print(script_list)
d = {}
for script in script_list:
    line_num = 0
    with open(script, encoding='utf-8-sig') as f:
        for line in f:
            if line_num == 0:
                key = line.split("\t")[0]
                line_num += 1

            elif line_num == 1:
                d[key] = line.strip()
                line_num -= 1

    break

print(d)



In [ ]:
archive_path = r'D:\IMDA - National Speech Corpus\PART1\DATA\CHANNEL0\WAVE\SPEAKER0001.zip'
from datasets.download import DownloadManager
dl_manager = DownloadManager()
if os.path.exists(archive_path):
    audio_files = dl_manager.iter_archive(archive_path)
    for path, f in audio_files:
        print(path)
        break
else:
    print('error')

### Exploring speaker information for train-test split

In [ ]:
import pandas as pd

p1_speaker_df = pd.read_excel("./Speaker Information (Part 1).XLSX", dtype={'SCD/PART1': object})
p2_speaker_df = pd.read_excel("./Speaker Information (Part 2).XLSX", dtype={'SCD/PART2': object})
p1_speaker_df.head()

In [ ]:
p1_speaker_df["SEX"].value_counts()

In [ ]:
p1_speaker_df["ACC"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X = p1_speaker_df[(p1_speaker_df["ACC"]=="CHINESE") & (p1_speaker_df["SEX"]=="F")]
print("number of rows: ", len(X))
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)

print("number of rows in train: ",len(X_train))
print("number of rows in test: ",len(X_test))

In [ ]:
genderlist = p1_speaker_df["SEX"].unique()
racelist = p1_speaker_df["ACC"].unique()
train_speaker_ids = []
test_speaker_ids = []

for gender in genderlist:
    for race in racelist:
        X = p1_speaker_df[(p1_speaker_df["ACC"]==race) & (p1_speaker_df["SEX"]==gender)]
        X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
        train_speaker_ids.extend(X_train["SCD/PART1"])
        test_speaker_ids.extend(X_test["SCD/PART1"])
        

In [ ]:
p1_speaker_df.loc[p1_speaker_df["SCD/PART1"]=="0001"].iloc[0]["SEX"]